In [1]:
import numpy as np

In [2]:
def abs_max_arg(l):
    return np.argmax(np.abs(l))

In [82]:
def upper_solve(A,b):
    A = A.astype('float64'); b = b.astype('float64')
    if (A.shape[1]!=b.shape[0]): raise Exception("Dimensions don't match")
    n = A.shape[1]
    x = np.zeros(n, dtype=np.float64)
    for i in range(n-1, -1, -1):
        tmp = b[i]
        tmp -= np.dot(x, A[i,:])
        x[i] = tmp/A[i,i]
    
    return x

def lower_solve(A,b):
    A = A.astype('float64'); b = b.astype('float64')
    if (A.shape[1]!=b.shape[0]): raise Exception("Dimensions don't match")
    n = A.shape[1]
    x = np.zeros(n, dtype=np.float64)
    for i in range(0,n):
        tmp = b[i]
        tmp -= np.dot(x, A[i,:])
        x[i] = tmp/A[i,i]
    
    return x

In [83]:
def lup(A):
    A = A.astype('float64')
    if A.shape[0] != A.shape[1]:
        raise Exception('Input is not a square matrix')
    m = A.shape[0]
    U = A.copy(); L = np.eye(m, dtype=np.float64); P = np.eye(m, dtype=np.float64)
    
    for k in range(m-1):
        i = k + abs_max_arg(U[k:,k])
        if(k!=i):
            U[[k,i], k:] = U[[i,k], k:]
            L[[k,i], :k] = L[[i,k], :k]
            P[[k,i], :] = P[[i,k], :]
        for j in range(k+1,m):
            L[j,k] = U[j,k]/U[k,k]
            U[j,k:] = U[j,k:] - L[j,k]*U[k,k:]
#         print(k,"\n\n",P,"\n\n",L,"\n\n",U,"\n-------------\n")
    return (P,L,U)

def solveLup(P,L,U,b):
    b_ = P@b
    y = lower_solve(L,b_)
    x = upper_solve(U,y)
    return x

In [138]:
def instabilityMatrix(m):
    M = 2*np.eye(m)-np.tri(m)
    last_col = np.ones(m)
    last_col[-1]=0
    M[:,-1] = M[:,-1]+last_col
    return M

In [139]:
A = np.array([[2,1,1,0], [4,3,3,1], [8,7,9,5],[6,7,9,8]])

In [140]:
(P,L,U) = lup(A)

In [141]:
A,(P,L,U)

(array([[2, 1, 1, 0],
        [4, 3, 3, 1],
        [8, 7, 9, 5],
        [6, 7, 9, 8]]),
 (array([[0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.]]),
  array([[ 1.        ,  0.        ,  0.        ,  0.        ],
         [ 0.75      ,  1.        ,  0.        ,  0.        ],
         [ 0.5       , -0.28571429,  1.        ,  0.        ],
         [ 0.25      , -0.42857143,  0.33333333,  1.        ]]),
  array([[ 8.        ,  7.        ,  9.        ,  5.        ],
         [ 0.        ,  1.75      ,  2.25      ,  4.25      ],
         [ 0.        ,  0.        , -0.85714286, -0.28571429],
         [ 0.        ,  0.        ,  0.        ,  0.66666667]])))

In [142]:
b = [4,3,2,1]

In [143]:
solveLup(P,L,U,b)

array([ 6.5, -5.5, -3.5,  4. ])

In [207]:
M = instabilityMatrix(60)
M[0,0]=1.1
(P,L,U)=lup(M)

In [208]:
P,L,U[:,-1]

(array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]),
 array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.90909091,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.90909091, -1.        ,  1.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-0.90909091, -1.        , -1.        , ...,  1.        ,
          0.        ,  0.        ],
        [-0.90909091, -1.        , -1.        , ..., -1.        ,
          1.        ,  0.        ],
        [-0.90909091, -1.        , -1.        , ..., -1.        ,
         -1.        ,  1.        ]]),
 array([1.00000000e+00, 1.90909091e+00, 3.81818182e+00, 7.63636364e+00,
        1.52727273e+01, 3.05454545e+01, 6.10909091e+

In [209]:
np.allclose(P@M,L@U)

False

In [210]:
(P@M), (L@U)

(array([[ 1.1,  0. ,  0. , ...,  0. ,  0. ,  1. ],
        [-1. ,  1. ,  0. , ...,  0. ,  0. ,  1. ],
        [-1. , -1. ,  1. , ...,  0. ,  0. ,  1. ],
        ...,
        [-1. , -1. , -1. , ...,  1. ,  0. ,  1. ],
        [-1. , -1. , -1. , ..., -1. ,  1. ,  1. ],
        [-1. , -1. , -1. , ..., -1. , -1. ,  1. ]]),
 array([[ 1.1,  0. ,  0. , ...,  0. ,  0. ,  1. ],
        [-1. ,  1. ,  0. , ...,  0. ,  0. ,  1. ],
        [-1. , -1. ,  1. , ...,  0. ,  0. ,  1. ],
        ...,
        [-1. , -1. , -1. , ...,  1. ,  0. , 16. ],
        [-1. , -1. , -1. , ..., -1. ,  1. , 32. ],
        [-1. , -1. , -1. , ..., -1. , -1. , 64. ]]))

In [211]:
np.linalg.norm(P@M-L@U,2)/np.linalg.norm(M,2)

1.9049271925203826

In [212]:
np.linalg.norm(M,2)

37.90592788935169

In [213]:
(L@U)[:,-1]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  2.,  4.,  8., 16., 32., 64.])